Builds 4 bee target files - 2 Columns each

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

Load input (first deploy to GitHub)

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/ModelEarth/RealityStream/main/input/bees/targets/bees-targets.csv")

In [ ]:
df.shape

(14567, 18)

In [ ]:
df.head()

,Unnamed: 0,Name,Fips,State,State ANSI,Ag District,Ag District Code,County,County ANSI,2002,2007,2012,2017,2022,2007_increase,2012_increase,2017_increase,2022_increase
0,29,Autauga County,1001.0,ALABAMA,1.0,BLACK BELT,40.0,Autauga County,1.0,212.0,201.0,119.0,137.0,258.0,0.0,0.0,1.0,1.0
1,30,Baldwin County,1003.0,ALABAMA,1.0,COASTAL PLAINS & GULF COAST,50.0,Baldwin County,3.0,1802.0,275.0,256.0,376.0,739.0,0.0,0.0,1.0,1.0
2,31,Baldwin County,1003.0,GEORGIA,13.0,CENTRAL,50.0,Baldwin County,9.0,3.0,517.0,517.0,42.0,656.0,1.0,1.0,0.0,1.0
3,32,Baldwin County,13009.0,ALABAMA,1.0,COASTAL PLAINS & GULF COAST,50.0,Baldwin County,3.0,1802.0,275.0,256.0,376.0,739.0,0.0,0.0,1.0,1.0
4,33,Baldwin County,13009.0,GEORGIA,13.0,CENTRAL,50.0,Baldwin County,9.0,3.0,517.0,517.0,42.0,656.0,1.0,1.0,0.0,1.0


In [3]:
df=df.groupby(["Fips"]).sum().reset_index()

In [4]:
df["2007_increase"]=(df["2002"]<df["2007"]).astype(int)
df["2012_increase"]=(df["2007"]<df["2012"]).astype(int)
df["2017_increase"]=(df["2012"]<df["2017"]).astype(int)
df["2022_increase"]=(df["2017"]<df["2022"]).astype(int)
df

,Fips,Unnamed: 0,Name,State,State ANSI,Ag District,Ag District Code,County,County ANSI,2002,2007,2012,2017,2022,2007_increase,2012_increase,2017_increase,2022_increase
0,1001.0,29,Autauga County,ALABAMA,1.0,BLACK BELT,40.0,Autauga County,1.0,212.0,201.0,119.0,137.0,258.0,0,0,1,1
1,1003.0,61,Baldwin CountyBaldwin County,ALABAMAGEORGIA,14.0,COASTAL PLAINS & GULF COASTCENTRAL,100.0,Baldwin CountyBaldwin County,12.0,1805.0,792.0,773.0,418.0,1395.0,0,0,0,1
2,1005.0,69,Barbour CountyBarbour County,ALABAMAWEST VIRGINIA,55.0,WIREGRASSNORTHWEST,80.0,Barbour CountyBarbour County,6.0,573.0,675.0,150.0,160.0,210.0,1,0,1,1
3,1007.0,77,Bibb CountyBibb County,ALABAMAGEORGIA,14.0,UPPER PLAINS & PIEDMONTCENTRAL,80.0,Bibb CountyBibb County,28.0,1034.0,526.0,574.0,478.0,34.0,0,1,0,0
4,1009.0,85,Blount CountyBlount County,ALABAMATENNESSEE,48.0,MOUNTAINS & EASTERN VALLEYEAST TENNESSEE,80.0,Blount CountyBlount County,18.0,506.0,762.0,1029.0,482.0,1125.0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3078,56037.0,14468,Sweetwater County,WYOMING,56.0,SOUTH CENTRAL,40.0,Sweetwater County,37.0,517.0,765.0,244.0,431.0,656.0,1,0,1,1
3079,56039.0,35043,Teton CountyTeton CountyTeton County,IDAHOMONTANAWYOMING,102.0,EASTNORTH CENTRALWEST,140.0,Teton CountyTeton CountyTeton County,219.0,1551.0,1052.0,1685.0,2694.0,694.0,0,1,1,0
3080,56041.0,14469,Uinta County,WYOMING,56.0,WEST,30.0,Uinta County,41.0,517.0,517.0,136.0,233.0,188.0,0,0,1,0
3081,56043.0,14470,Washakie County,WYOMING,56.0,NORTHWEST,10.0,Washakie County,43.0,6.0,517.0,517.0,13.0,32.0,1,0,0,1


In [5]:
df_2007 = df[["Fips","2007_increase"]]
df_2007 = df_2007.rename(columns={'2007_increase': 'Target'})
df_2007['Fips'] = df_2007['Fips'].astype(int)

df_2012 = df[["Fips","2012_increase"]]
df_2012 = df_2012.rename(columns={'2012_increase': 'Target'})
df_2012['Fips'] = df_2012['Fips'].astype(int)

df_2017 = df[["Fips","2017_increase"]]
df_2017 = df_2017.rename(columns={'2017_increase': 'Target'})
df_2017['Fips'] = df_2017['Fips'].astype(int)

df_2022 = df[["Fips","2022_increase"]]
df_2022 = df_2022.rename(columns={'2022_increase': 'Target'})
df_2022['Fips'] = df_2022['Fips'].astype(int)

In [6]:
df_2007.to_csv('bees-targets2007.csv', index=False)
df_2012.to_csv('bees-targets2012.csv', index=False)
df_2017.to_csv('bees-targets2017.csv', index=False)
df_2022.to_csv('bees-targets2022.csv', index=False)

In [ ]:
df_2007.to_csv('bees-targets2007.csv', index=False)

In [ ]:
df_2012 = df.iloc[:,[2,-3]]
df_2012 = df_2012.drop_duplicates()
df_2012 = df_2012.rename(columns={'2012_increase': 'Target'})
df_2012 = df_2012.dropna(subset=["Fips", "Target"]).reset_index(drop=True)
df_2012['Fips'] = df_2012['Fips'].astype(int)
df_2012['Target'] = df_2012['Target'].astype(int)
df_2012

,Fips,Target
0,1001,0
1,1003,0
2,1003,1
3,13009,0
4,13009,1
...,...,...
4199,56035,1
4200,56037,0
4201,56041,0
4202,56043,1


In [ ]:
df_2012.to_csv('bees-targets2012.csv', index=False)

In [ ]:
df_2017 = df.iloc[:,[2,-2]]
df_2017 = df_2017.drop_duplicates()
df_2017 = df_2017.rename(columns={'2017_increase': 'Target'})
df_2017 = df_2017.dropna(subset=["Fips", "Target"]).reset_index(drop=True)
df_2017['Fips'] = df_2017['Fips'].astype(int)
df_2017['Target'] = df_2017['Target'].astype(int)
df_2017

,Fips,Target
0,1001,1
1,1003,1
2,1003,0
3,13009,1
4,13009,0
...,...,...
4221,56035,0
4222,56037,1
4223,56041,1
4224,56043,0


In [ ]:
df_2017.to_csv('bees-targets2017.csv', index=False)

In [ ]:
df_2022 = df.iloc[:,[2,-1]]
df_2022 = df_2022.drop_duplicates()
df_2022 = df_2022.rename(columns={'2022_increase': 'Target'})
df_2022 = df_2022.dropna(subset=["Fips", "Target"]).reset_index(drop=True)
df_2022['Fips'] = df_2022['Fips'].astype(int)
df_2022['Target'] = df_2022['Target'].astype(int)
df_2022

,Fips,Target
0,1001,1
1,1003,1
2,13009,1
3,1005,1
4,54001,1
...,...,...
4167,56035,1
4168,56037,1
4169,56041,0
4170,56043,1


In [ ]:
df_2022.to_csv('bees-targets2022.csv', index=False)

Update

In [ ]:
df = df.dropna(subset=["2007_increase", "2012_increase", "2017_increase", "2022_increase"])
df.head()

,Unnamed: 0,Name,Fips,State,State ANSI,Ag District,Ag District Code,County,County ANSI,2002,2007,2012,2017,2022,2007_increase,2012_increase,2017_increase,2022_increase
0,29,Autauga County,1001.0,ALABAMA,1.0,BLACK BELT,40.0,Autauga County,1.0,212.0,201.0,119.0,137.0,258.0,0.0,0.0,1.0,1.0
1,30,Baldwin County,1003.0,ALABAMA,1.0,COASTAL PLAINS & GULF COAST,50.0,Baldwin County,3.0,1802.0,275.0,256.0,376.0,739.0,0.0,0.0,1.0,1.0
2,31,Baldwin County,1003.0,GEORGIA,13.0,CENTRAL,50.0,Baldwin County,9.0,3.0,517.0,517.0,42.0,656.0,1.0,1.0,0.0,1.0
3,32,Baldwin County,13009.0,ALABAMA,1.0,COASTAL PLAINS & GULF COAST,50.0,Baldwin County,3.0,1802.0,275.0,256.0,376.0,739.0,0.0,0.0,1.0,1.0
4,33,Baldwin County,13009.0,GEORGIA,13.0,CENTRAL,50.0,Baldwin County,9.0,3.0,517.0,517.0,42.0,656.0,1.0,1.0,0.0,1.0


In [ ]:
df_2022 = df.iloc[:, list(range(9)) + [13, 17]]
df_2022 = df_2022.reset_index(drop=True)
df_2022.head()

,Unnamed: 0,Name,Fips,State,State ANSI,Ag District,Ag District Code,County,County ANSI,2022,2022_increase
0,29,Autauga County,1001.0,ALABAMA,1.0,BLACK BELT,40.0,Autauga County,1.0,258.0,1.0
1,30,Baldwin County,1003.0,ALABAMA,1.0,COASTAL PLAINS & GULF COAST,50.0,Baldwin County,3.0,739.0,1.0
2,31,Baldwin County,1003.0,GEORGIA,13.0,CENTRAL,50.0,Baldwin County,9.0,656.0,1.0
3,32,Baldwin County,13009.0,ALABAMA,1.0,COASTAL PLAINS & GULF COAST,50.0,Baldwin County,3.0,739.0,1.0
4,33,Baldwin County,13009.0,GEORGIA,13.0,CENTRAL,50.0,Baldwin County,9.0,656.0,1.0


In [ ]:
states = ["AK", "AL", "AR", "AZ", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "IA",
          "ID", "IL", "IN", "KS", "KY", "LA", "MA", "MD", "ME", "MI", "MN", "MO",
          "MS", "MT", "NC", "ND", "NE", "NH", "NJ", "NM", "NV", "NY", "OH", "OK",
          "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VA", "VT", "WA", "WI",
          "WV", "WY"]
naics_value = 6
df_full = pd.DataFrame()
for state in states:
  url = f"https://raw.githubusercontent.com/ModelEarth/community-timelines/main/training/all-years/naics{naics_value}/{state}-all-years.csv"
  #https://github.com/ModelEarth/community-timelines/blob/main/training/all-years/naics6/AK-all-years.csv
  df_ind = pd.read_csv(url)
  df_full = pd.concat([df_full, df_ind], axis=0, join='outer')

In [ ]:
df_full.head()

,Unnamed: 0,Fips,County,Emp-113310-2017,Emp-113310-2018,Emp-113310-2019,Emp-113310-2020,Emp-113310-2021,Emp-114111-2017,Emp-114111-2018,...,Emp-212391-2019,Emp-212391-2020,Est-212391-2017,Est-212391-2018,Est-212391-2019,Est-212391-2020,Pay-212391-2017,Pay-212391-2018,Pay-212391-2019,Pay-212391-2020
0,0,2016,Aleutians West Census Area,NaN,NaN,NaN,NaN,NaN,6.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2060,Bristol Bay Borough,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2150,Kodiak Island Borough,NaN,NaN,NaN,NaN,NaN,28.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2188,Northwest Arctic Borough,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2050,Bethel Census Area,NaN,NaN,NaN,NaN,NaN,0.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_full = df_full.loc[:, df_full.isnull().mean() < .5]

In [ ]:
df_full = df_full.merge(df_2022[['County','2022','2022_increase']], how='inner', on='County')
#df_full = df_full.drop('County', axis=1)
df_full.head()

,Unnamed: 0,Fips,County,Emp-236115-2017,Emp-236115-2018,Emp-236115-2019,Emp-236115-2020,Emp-236115-2021,Emp-236118-2017,Emp-236118-2018,...,Population-2019,Population-2020,Population-2021,UrbanDensity-2017,UrbanDensity-2018,UrbanDensity-2019,UrbanDensity-2020,UrbanDensity-2021,2022,2022_increase
0,0,1099,Monroe County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21,21.0,20.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0
1,0,1099,Monroe County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21,21.0,20.0,0.0,0.0,0.0,0.0,0.0,34.0,0.0
2,0,1099,Monroe County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21,21.0,20.0,0.0,0.0,0.0,0.0,0.0,99.0,0.0
3,0,1099,Monroe County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21,21.0,20.0,0.0,0.0,0.0,0.0,0.0,114.0,0.0
4,0,1099,Monroe County,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21,21.0,20.0,0.0,0.0,0.0,0.0,0.0,168.0,1.0


In [ ]:
df_full.shape

(183159, 617)

In [ ]:
#df_full_model = df_full.loc[:, df_full.isnull().mean() < .5]
#df_full_model = df_full_model.dropna(subset=["Name"])
#df_full_model.shape

(183224, 151)

In [ ]:
df_full_model = df_full.rename(columns={'2022_increase': 'label'})

In [ ]:
df_full_model = df_full_model.fillna(df_full_model.median(numeric_only=True))

In [ ]:
df_full_model.head()

,Unnamed: 0,Fips,County,Emp-236115-2017,Emp-236115-2018,Emp-236115-2019,Emp-236115-2020,Emp-236115-2021,Emp-236118-2017,Emp-236118-2018,...,Population-2019,Population-2020,Population-2021,UrbanDensity-2017,UrbanDensity-2018,UrbanDensity-2019,UrbanDensity-2020,UrbanDensity-2021,2022,label
0,0,1099,Monroe County,30.0,28.0,31.0,31.0,33.0,33.0,32.0,...,21,21.0,20.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0
1,0,1099,Monroe County,30.0,28.0,31.0,31.0,33.0,33.0,32.0,...,21,21.0,20.0,0.0,0.0,0.0,0.0,0.0,34.0,0.0
2,0,1099,Monroe County,30.0,28.0,31.0,31.0,33.0,33.0,32.0,...,21,21.0,20.0,0.0,0.0,0.0,0.0,0.0,99.0,0.0
3,0,1099,Monroe County,30.0,28.0,31.0,31.0,33.0,33.0,32.0,...,21,21.0,20.0,0.0,0.0,0.0,0.0,0.0,114.0,0.0
4,0,1099,Monroe County,30.0,28.0,31.0,31.0,33.0,33.0,32.0,...,21,21.0,20.0,0.0,0.0,0.0,0.0,0.0,168.0,1.0


Build Model

In [ ]:
df_full_model.label.value_counts()

label
1.0    109890
0.0     73269
Name: count, dtype: int64

In [ ]:
X = df_full_model.drop(columns=['Fips','County','label','2022'])
y = df_full_model['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
n_estimator = 100
depth = 50

opt_rfc = RandomForestClassifier(n_estimators= n_estimator, max_depth=depth)
opt_rfc.fit(X_train, y_train)
pred_train = opt_rfc.predict(X_train)
pred_test = opt_rfc.predict(X_test)

accu_train = accuracy_score(y_train, pred_train)
accu_test = accuracy_score(y_test, pred_test)

f1_train = f1_score(y_train, pred_train)
f1_test = f1_score(y_test, pred_test)

recall_train = recall_score(y_train, pred_train)
recall_test = recall_score(y_test, pred_test)

precision_train = precision_score(y_train, pred_train)
precision_test = precision_score(y_test, pred_test)
print(f'Accuracy score on train set = {accu_train} and accuracy score on test set = {accu_test}')
print(f'f1 score on train set = {f1_train} and f1 score on test set = {f1_test}')
print(f'Recall score on train set = {recall_train} and recall score on test set = {recall_test}')
print(f'Precision score on train set = {precision_train} and precision score on test set = {precision_test}')

Accuracy score on train set = 0.6192169361278126 and accuracy score on test set = 0.6010864817645775
f1 score on train set = 0.7371198379231548 and f1 score on test set = 0.7243818253833532
Recall score on train set = 0.8898102648102648 and recall score on test set = 0.8737373737373737
Precision score on train set = 0.6291571827270315 and precision score on test set = 0.6186334203150672


In [ ]:
# a dict to hold feature_name: feature_importance
features = {}
for feature, importance in zip(X.columns, opt_rfc.feature_importances_):
    features[feature] = importance

importances = pd.DataFrame.from_dict(features, orient='index').rename(columns={0: 'Gini-importance'})
importances.sort_values(by='Gini-importance', ascending=False).head(10)

,Gini-importance
Latitude-2017,0.025665
Longitude-2018,0.024310
Longitude-2020,0.023818
Longitude-2017,0.023814
Longitude-2019,0.022715
Latitude-2018,0.022701
Longitude-2021,0.022146
Latitude-2021,0.021612
Latitude-2019,0.021514
Latitude-2020,0.021320


In [ ]:
rfc=RandomForestClassifier()
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth' : [10, 20, 50]
}
# param_grid = {
#     'n_estimators': [50, 100, 150, 200, 250],
#     'max_depth' : [5, 10, 20, 50, 80]
# }
rfcs = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 3)
rfcs.fit(X_train, y_train)
#save optimal parameters
n_estimator = rfcs.best_params_['n_estimators']
depth = rfcs.best_params_['max_depth']
print(f'n_estimators = {n_estimator} and max_depth = {depth}')

n_estimators = 100 and max_depth = 20


In [ ]:
#fit a new random forest with optimal parameters
opt_rfc = RandomForestClassifier(n_estimators= n_estimator, max_depth=depth)
opt_rfc.fit(X_train, y_train)
pred_train = opt_rfc.predict(X_train)
pred_test = opt_rfc.predict(X_test)

accu_train = accuracy_score(y_train, pred_train)
accu_test = accuracy_score(y_test, pred_test)

f1_train = f1_score(y_train, pred_train)
f1_test = f1_score(y_test, pred_test)

recall_train = recall_score(y_train, pred_train)
recall_test = recall_score(y_test, pred_test)

precision_train = precision_score(y_train, pred_train)
precision_test = precision_score(y_test, pred_test)
print(f'Accuracy score on train set = {accu_train} and accuracy score on test set = {accu_test}')
print(f'f1 score on train set = {f1_train} and f1 score on test set = {f1_test}')
print(f'Recall score on train set = {recall_train} and recall score on test set = {recall_test}')
print(f'Precision score on train set = {precision_train} and precision score on test set = {precision_test}')

Accuracy score on train set = 0.9591346645836034 and accuracy score on test set = 0.9527629963160049
f1 score on train set = 0.963413144392866 and f1 score on test set = 0.9577629748920284
Recall score on train set = 0.9697509990777744 and recall score on test set = 0.9652764115679716
Precision score on train set = 0.9571575945142302 and precision score on test set = 0.9503655997288267


In [ ]:
# a dict to hold feature_name: feature_importance
features = {}
for feature, importance in zip(X.columns, opt_rfc.feature_importances_):
    features[feature] = importance

importances = pd.DataFrame.from_dict(features, orient='index').rename(columns={0: 'Gini-importance'})
importances.sort_values(by='Gini-importance', ascending=False).head(10)

,Gini-importance
2017,0.756043
Km2,0.005089
Population,0.002960
Pay-4471,0.002919
Pay-8131,0.002808
Emp-4471,0.002748
Emp-8131,0.002460
Emp-5221,0.002383
Pay-7225,0.002363
Pay-5617,0.002342
